In [2]:
with open ('../GPT/data/wikitext-103/wiki.train.tokens', 'r') as f:
    text = f.read()

print(text[:1000])

 
 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomer

In [4]:
import tiktoken
tokenizer = tiktoken.get_encoding("o200k_base")
tokens = tokenizer.encode(text)
print(tokens[:100])

[793, 314, 142393, 131854, 109152, 18857, 314, 56319, 8675, 73, 16191, 860, 142393, 131854, 220, 18, 712, 464, 4060, 29, 109152, 350, 18938, 712, 15882, 99, 23868, 3385, 93460, 69686, 7052, 18368, 19913, 74066, 18, 1366, 11980, 887, 142393, 131854, 328, 290, 147762, 220, 18, 1546, 1366, 22378, 22653, 316, 472, 142393, 131854, 109152, 18857, 7539, 10198, 1366, 382, 261, 84570, 5430, 759, 12, 31, 8252, 3823, 2813, 9742, 656, 165189, 326, 10260, 5970, 3625, 395, 290, 9103, 23191, 61768, 887, 93105, 306, 8685, 220, 667, 16, 306, 10198, 1366, 480, 382, 290, 6914, 2813, 306, 290, 142393, 131854, 5594]


# Scaled Dot-Product 
takes in a query key and value
- Query : current word similarities are being calculated on
- Key : all tokens in the sequence which will be computed with the query to provide similarities ( attention scores)
- Value : the actual token 

## Method
- Query and Key are dot product multiplied (matrix multiplication)
- the value is then scaled 
- Masked if in decoder
- softmax to calulate probs (attention weights)
- matmul of attention weights with value

```mermaid
graph TD 
Q --> MM1[MatMul] 
K(K) --> MM1 
MM1 --> Scale[Scale] 
Scale --> Mask[Mask opt.]
Mask --> SoftMax[SoftMax]
SoftMax --> MM2[MatMul]
V ------> MM2
MM2 --> Output
```

# Multi-Head Attention 

Scaling the K,Q,V to multiple scaled Dot-Product Attention heads using linear projection.

Once all of the values have been computed then concatenating them.  

(optional addition of dropout or other reg techniques)

linearly project them 

In [8]:
import torch
train_tokens = torch.tensor(tokens[:50000], dtype=torch.long, device='mps')
torch.set_printoptions(linewidth=140)


In [9]:
train_tokens[:10]

tensor([   793,    314, 142393, 131854, 109152,  18857,    314,  56319,   8675,     73], device='mps:0')

In [ ]:
BLOCK_SIZE = 32 # number of tokens in each sequence
BATCH_SIZE = 4 # number of parallel processes in a head

def get_batch(tokens):
    ix = torch.randint(len(tokens) - BLOCK_SIZE, (BATCH_SIZE,)) # random starting index for each sequence
    try :
        x = torch.stack([tokens[i:i+BLOCK_SIZE] for i in ix]) # stack the sequences into a
        #batch makes it a 2d tensor in this case matrix which is batchsize x block_size
        y = torch.stack([tokens[i+1:i+BLOCK_SIZE+1] for i in ix]) #shifted by one token
        return x, y
    except :
        seq_len = tokens.shape[0] if isinstance(tokens, torch.Tensor) else len(tokens)
        pad_token = 199999
        if seq_len < BLOCK_SIZE:
            pad_length = BLOCK_SIZE - seq_len
            # Pad x and y with pad_token
            x = torch.nn.functional.pad(x, (0, pad_length), value=pad_token)
            y = torch.nn.functional.pad(y, (0, pad_length), value=pad_token)
        print(f"Sequence length: {seq_len}")
        return x, y


In [65]:
tokenizer.eot_token

199999

In [81]:
current, target = get_batch(train_tokens)

In [ ]:
current.shape # currently batch_szie x block_size

torch.Size([4, 32])

# B -> Batch_size (parallel computations in a head)
# T -> Block_size (sequence length)
# C -> Number of Embeddings

# C the embeddings
Extracted by transforming the tokens to a look up vector of vector representations. 
This extracts semantic and syntatic information about the tokens.
This is the initial input feature for the transformer.
Usually between 128 and 1024 but depending on hardware, scale to as needed
Takes 2 values, the vocabulary size (ie number of unique tokens) and the embedding dim (based on compuational and memory constraints larger is better though)


In [89]:
EMBEDDING_DIM = 128

In [84]:
import torch.nn as nn
import math
import torch.nn.functional as F

In [101]:
class MultiHeadAttention(nn.Module):
    def __init__(self, block_size:int, embedding_dim:int, n_heads:int, dropout:float=0.1):
        """
        Args:
            block_size (int): Number of tokens in each sequence.
            embedding_dim (int): Dimension of the embedding space.
            n_heads (int): Number of attention heads.
        """
        super().__init__()

        assert embedding_dim % n_heads == 0, "Embedding dimension must be divisible by number of heads"

        self.head_dim = embedding_dim // n_heads # number of parallel processes is the number of
        # embeddings floor div by number of heads ie each head will work on a subset of the embeddings
        self.block_size = block_size
        self.embedding_dim = embedding_dim
        self.dropout = dropout
        self.n_heads = n_heads

        self.qkv_projection = nn.Linear(embedding_dim, 3*embedding_dim, device='mps') # has the projection for all three stacked has
        self.output_projection = nn.Linear(embedding_dim, embedding_dim, device='mps') # has the projection for all three stacked has
        self.dropout_layer = nn.Dropout(dropout)


    def forward(self, x, use_causal_mask: bool = False):

        batch_size, block_size, embedding_dim = x.shape # parallel processes within a head, sequence length, embedding dimension
        print(f"batch_size: {batch_size}, block_size: {block_size}, embedding_dim: {embedding_dim}")

        qkv = self.qkv_projection(x)  # (batch_size, block_size, 3 * embedding_dim)
        query, key, value = qkv.split(self.embedding_dim, dim=2)  # (batch_size, block_size, embedding_dim) each
        print(f"query.shape: {query.shape}, key.shape: {key.shape}, value.shape: {value.shape}")
        print(f"query: {query[:2]}, key: {key[:2]}, value: {value[:2]}")

        # reshape to (batch_size, block_size, n_heads, embedding_dim // n_heads) this allows parallel processing as the tensor is
        # split into its n_heads
        query = query.view(batch_size, block_size, self.n_heads,self.head_dim).transpose(1, 2) # (batch_size, n_heads, block_size, embedding_dim // n_heads)
        key = key.view(batch_size, block_size, self.n_heads,self.head_dim).transpose(1, 2) # (batch_size, n_heads, block_size, embedding_dim // n_heads)
        value = value.view(batch_size, block_size, self.n_heads,self.head_dim).transpose(1, 2) # (batch_size, n_heads, block_size, embedding_dim // n_heads)
        print(f"query.shape: {query.shape}, key.shape: {key.shape}, value.shape: {value.shape}")
        print(f"query: {query[:2]}, key: {key[:2]}, value: {value[:2]}")

        attention_scores = torch.matmul(query, key.transpose(-2,-1))
        print(f'attentions{attention_scores[:2]}')
        scaled_attention_scores = attention_scores / math.sqrt(self.head_dim) # scaling
        print(f'scaled_attention_scores{scaled_attention_scores[:2]}')

        if use_causal_mask:
            tril = torch.tril(torch.ones(block_size, block_size, device=scaled_attention_scores.device)).view(1, 1, block_size, block_size)
            print(f'tril{tril[:2]}')
            scaled_attention_scores = scaled_attention_scores.masked_fill(tril == 0, float('-inf'))
            print(f'scaled_attention_scores{scaled_attention_scores[:2]}')

        attention_weights = F.softmax(scaled_attention_scores, dim=-1)
        print(f'attention_weights{attention_weights[:2]}')
        attention_weights = self.dropout_layer(attention_weights)
        print(f'attention_weights{attention_weights[:2]}')

        context_vectors = torch.matmul(attention_weights, value) # (batch_size, n_heads, block_size, embedding_dim // n_heads)
        print(f'context_vectors{context_vectors[:2]}')
        context_vectors = context_vectors.transpose(1, 2).contiguous().view(batch_size, block_size,embedding_dim) # (batch_size, block_size, embedding_dim)
        print(f'context_vectors{context_vectors[:2]}')

        output = self.output_projection(context_vectors) # (batch_size, block_size, embedding_dim)
        print(f'output{output[:2]}')
        output = self.dropout_layer(output)
        print(f'output{output[0][0][0]}')
        return output


In [85]:
x, y = get_batch(train_tokens)

In [98]:
mha = MultiHeadAttention(block_size=BLOCK_SIZE, embedding_dim=EMBEDDING_DIM, n_heads=4)

In [94]:
# mha.forward(x, use_causal_mask=True)
embedding_layer = nn.Embedding(tokenizer.n_vocab, EMBEDDING_DIM, device='mps')
x_embedded = embedding_layer(x)
x_embedded.shape

torch.Size([4, 32, 128])

In [102]:
mha(x_embedded, use_causal_mask=True)

batch_size: 4, block_size: 32, embedding_dim: 128
query.shape: torch.Size([4, 32, 128]), key.shape: torch.Size([4, 32, 128]), value.shape: torch.Size([4, 32, 128])
query: tensor([[[ 0.0734, -0.5131, -1.5445,  ..., -0.1679,  0.5132, -0.2483],
         [-0.2512,  0.3565,  0.3393,  ...,  0.0423, -0.3175,  0.3175],
         [-0.0335, -0.6852,  0.7163,  ...,  0.0236,  0.3435, -0.0236],
         ...,
         [-0.7947,  0.1875, -0.5283,  ..., -0.5621, -0.2579, -0.1302],
         [-0.7005,  0.2542, -0.4663,  ..., -0.2459, -0.2931,  0.4226],
         [ 0.2155,  0.9124,  0.4845,  ...,  0.2412, -0.6053, -0.2347]],

        [[-0.1661, -0.4890, -0.5011,  ...,  0.1103, -0.2260, -0.5821],
         [-0.4166, -0.7103,  0.6181,  ..., -0.3068, -0.1211, -0.1450],
         [-0.2102, -0.3381, -0.3500,  ...,  0.5178,  0.3451,  1.0514],
         ...,
         [ 0.0913, -0.9749, -0.6948,  ...,  0.6463,  0.4499, -0.6304],
         [-0.7916, -0.8350,  0.5759,  ...,  0.3318,  0.4120,  1.6871],
         [ 0.7143,

tensor([[[-0.0000e+00,  7.0358e-01,  7.7160e-02,  ...,  6.3563e-01, -4.4907e-01,  2.7774e-01],
         [ 2.1755e-01,  2.6551e-01, -5.4010e-01,  ...,  4.9836e-01, -2.1173e-01,  3.0570e-01],
         [ 9.8642e-02,  7.6984e-02, -1.2861e-01,  ..., -3.1456e-01, -3.3882e-01,  5.6492e-01],
         ...,
         [ 6.8187e-02,  8.5039e-02, -2.1376e-01,  ..., -6.2611e-02, -1.2242e-01, -0.0000e+00],
         [ 0.0000e+00,  1.3702e-01, -2.3812e-01,  ..., -0.0000e+00, -1.1605e-01, -1.9688e-02],
         [ 0.0000e+00,  1.6711e-01, -2.3819e-01,  ..., -1.1501e-01, -5.6460e-02, -1.4734e-01]],

        [[-1.7896e-01,  2.9032e-01,  5.7247e-01,  ...,  3.4801e-02,  3.8493e-01, -2.3372e-02],
         [-1.4554e-01, -2.1643e-01,  8.2095e-01,  ...,  3.8979e-02,  9.5645e-02,  2.6830e-01],
         [-1.2423e-01, -1.7814e-01,  3.8735e-01,  ...,  7.0171e-02,  5.1214e-01,  7.9267e-02],
         ...,
         [ 3.9976e-02, -2.9452e-02,  2.3572e-02,  ..., -1.0025e-01, -0.0000e+00,  0.0000e+00],
         [ 2.8194e-0